In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [7]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME', and other uneccessary columns
application_df = application_df.drop(columns=['EIN','NAME', 'STATUS','SPECIAL_CONSIDERATIONS'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [9]:
# Determine which values to replace if counts are less than ...?
type_counts = application_df.APPLICATION_TYPE.value_counts()
replace_application = list(type_counts[type_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [10]:
classification_counts = application_df.CLASSIFICATION.value_counts()
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [12]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop(columns=['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [43]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 70
hidden_nodes_layer3 = 50
hidden_nodes_layer4 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 100)               4100      
_________________________________________________________________
dense_20 (Dense)             (None, 70)                7070      
_________________________________________________________________
dense_21 (Dense)             (None, 50)                3550      
_________________________________________________________________
dense_22 (Dense)             (None, 30)                1530      
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 31        
Total params: 16,281
Trainable params: 16,281
Non-trainable params: 0
_________________________________________________________________


In [44]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

In [45]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

In [49]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
184/804 [=====>........................] - ETA: 0s - loss: 0.6907 - accuracy: 0.5363
Epoch 00001: saving model to checkpoints/weights.01.hdf5
804/804 [==============================] - 1s 862us/step - loss: 0.6912 - accuracy: 0.5324
Epoch 2/100
365/804 [============>.................] - ETA: 0s - loss: 0.6916 - accuracy: 0.5285
Epoch 00002: saving model to checkpoints/weights.02.hdf5
804/804 [==============================] - 1s 853us/step - loss: 0.6911 - accuracy: 0.5324
Epoch 3/100
589/804 [====================>.........] - ETA: 0s - loss: 0.6916 - accuracy: 0.5292
Epoch 00003: saving model to checkpoints/weights.03.hdf5
804/804 [==============================] - 1s 879us/step - loss: 0.6911 - accuracy: 0.5324
Epoch 4/100
787/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5326
Epoch 00004: saving model to checkpoints/weights.04.hdf5
804/804 [==============================] - 1s 849us/step - loss: 0.6912 - accuracy: 0.5324
Epoch 5/100
804/804 

446/804 [===============>..............] - ETA: 0s - loss: 0.6911 - accuracy: 0.5324
Epoch 00038: saving model to checkpoints/weights.38.hdf5
804/804 [==============================] - 1s 894us/step - loss: 0.6911 - accuracy: 0.5324
Epoch 39/100
623/804 [======================>.......] - ETA: 0s - loss: 0.6908 - accuracy: 0.5344
Epoch 00039: saving model to checkpoints/weights.39.hdf5
804/804 [==============================] - 1s 912us/step - loss: 0.6911 - accuracy: 0.5324
Epoch 40/100
804/804 [==============================] - 1s 848us/step - loss: 0.6911 - accuracy: 0.5324
Epoch 41/100
  1/804 [..............................] - ETA: 1s - loss: 0.6872 - accuracy: 0.5625
Epoch 00041: saving model to checkpoints/weights.41.hdf5
804/804 [==============================] - 1s 937us/step - loss: 0.6911 - accuracy: 0.5324
Epoch 42/100
180/804 [=====>........................] - ETA: 0s - loss: 0.6918 - accuracy: 0.5276
Epoch 00042: saving model to checkpoints/weights.42.hdf5
804/804 [=======

669/804 [=======================>......] - ETA: 0s - loss: 0.6909 - accuracy: 0.5343
Epoch 00075: saving model to checkpoints/weights.75.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 76/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 77/100
 88/804 [==>...........................] - ETA: 0s - loss: 0.6898 - accuracy: 0.5430
Epoch 00077: saving model to checkpoints/weights.77.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 78/100
266/804 [========>.....................] - ETA: 0s - loss: 0.6904 - accuracy: 0.5371
Epoch 00078: saving model to checkpoints/weights.78.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 79/100
485/804 [=================>............] - ETA: 0s - loss: 0.6908 - accuracy: 0.5347
Epoch 00079: saving model to checkpoints/weights.79.hdf5
804/804 [===============

In [50]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6876 - accuracy: 0.5467
Loss: 0.6875672936439514, Accuracy: 0.5467055439949036


In [51]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")